# Tests
In the following notebook there will be several tests about the source code and the usabilty of the Knowledge Graph. In first place it's required to import all necessary libraries and connect to the database.

In [1]:
# importing necessary libraries
import os
import timeit
import datetime as dt
import pandas as pd
from termcolor import colored
from helpers.helper_functions import init_connection, excel_import, export_to_excel, test_query, reset_db

# init connection to the neo4j database
graph = init_connection()

Connected to the database


## Functional Tests

### Test Creation of Knowledge Graph
This test case checks if the Knowledge Graph got created successfully by validating the number of nodes and relationships.

In [2]:
# query to get the number of nodes in the database
query_node_count = """
MATCH (n)
RETURN count(n) as node_count
"""
# run the query and get the data of the result
node_result =  graph.run(query_node_count).data()
# extract the number of nodes from the result
number_of_nodes = node_result[0]['node_count']
# assert the number of nodes is at least 1
assert(number_of_nodes > 0)
# print the number of nodes
print(f"Number of nodes in the database: {number_of_nodes}")


# query to get the number of relationships in the database
query_node_count = """
MATCH ()-[r]->()
RETURN count(r) as relationship_count
"""
# run the query and get the data of the result
node_result = graph.run(query_node_count).data()
# extract the number of relationships from the result
number_of_rels = node_result[0]['relationship_count']
# assert the number of relationships is at least 1
assert(number_of_rels > 0)
# print the number of relationships
print(f"Number of relationships in the database: {node_result[0]['relationship_count']}")

Number of nodes in the database: 380
Number of relationships in the database: 602


### Get all Doctors
This query returns all doctors and tests if all nodes have a specialization attribute to be sure there are only doctor nodes returned.

In [3]:
# query to get all Doctor nodes
query_node_count = """
MATCH (d:Doctor)
RETURN d.name as name, d.specialization as specialization, d.yearsOfExperience as years_of_experience, d.contactEmail as contact_email
"""
# run the query and get the data of the result
node_result = graph.run(query_node_count).data()
# extract the number of doctors from the result
number_of_doctors = len(node_result)
# assert that all doctors have a specialization to be sure that there are only doctors returned
assert all(doctor['specialization'] for doctor in node_result), node_result
# print the number of doctors
print(f"Number of doctors in the database: {number_of_doctors}")
# print the details of the doctors
pd.DataFrame(node_result)

Number of doctors in the database: 10


,name,specialization,years_of_experience,contact_email
0,Dr. Emily Carter,Cardiology,15,emily.carter@example.com
1,Dr. Michael Johnson,Dermatology,18,michael.johnson@example.com
2,Dr. Sophia Lee,Neurology,12,sophia.lee@example.com
3,Dr. Benjamin Smith,Pediatrics,20,benjamin.smith@example.com
4,Dr. Olivia Brown,Oncology,22,olivia.brown@example.com
5,Dr. Ethan Davis,Orthopedics,17,ethan.davis@example.com
6,Dr. Ava Wilson,Psychiatry,14,ava.wilson@example.com
7,Dr. Liam Martinez,General Surgery,25,liam.martinez@example.com
8,Dr. Isabella Garcia,Radiology,10,isabella.garcia@example.com
9,Dr. Noah Miller,Ophthalmology,13,noah.miller@example.com


### Test Export Functionality
This test case checks if the export functionality works correctly. This is done by getting the number of files in the export directory. Afterwards run the export script and check subsequently if the number of files have increased.

In [4]:
export_path = "../data/export"

# number of files in export directory
number_of_files = len(os.listdir(export_path))

# define the current time for the filename
current_time = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

export_to_excel(current_time=current_time, graph=graph, export_path=export_path)

# check if the number of files in the export directory has increased
number_of_files_after = len(os.listdir(export_path))

# assert that the number of files has increased by 1
assert number_of_files_after - number_of_files == 1, "Export failed: no new file created."

Export completed. File saved at: ../data/export/export_2025-04-18_20-13-19.xlsx


### Test Import Functionality
This test case checks if the import functionality works correctly. This is done by getting the number of nodes and relationships. Afterwards run the import script and check subsequently if the number of nodes and relationships have increased. 

In [5]:
# query to get the number of current nodes
query_node_count = """
MATCH (n)
RETURN count(n) as node_count
"""
# query to get the number of current relationships
query_rel_count = """
MATCH ()-[r]->()
RETURN count(r) as relationship_count
"""

# run the query and get the data of the result
node_result = graph.run(query_node_count).data()
# run the query and get the data of the result
rel_result = graph.run(query_rel_count).data()

# extract the number of nodes from the result
number_of_nodes = node_result[0]['node_count']
# extract the number of relationships from the result
number_of_rels = rel_result[0]['relationship_count']

# define the excel file path
import_file = pd.ExcelFile("../data/import/import_data.xlsx")

# import the data from the excel file into the database
excel_import(excel_file=import_file, graph=graph)

# query to get the number of nodes after importing the data
after_result = graph.run(query_node_count).data()
# query to get the number of relationships after importing the data
after_rel_result = graph.run(query_rel_count).data()

# extract the number of nodes from the result
after_number_of_nodes = after_result[0]['node_count']
# extract the number of relationships from the result
after_number_of_rels = after_rel_result[0]['relationship_count']

# assert that the number of nodes has increased
assert after_number_of_nodes > number_of_nodes, f"Number of nodes before import: {number_of_nodes}, after import: {after_number_of_nodes}"
# assert that the number of relationships has increased
assert after_number_of_rels > number_of_rels, f"Number of relationships before import: {number_of_rels}, after import: {after_number_of_rels}"

Node worksheets:  ['Doctor']
Relationship worksheets:  ['REL_Doctor'] 

Created 1 nodes with the label 'Doctor'
All nodes have been created successfully. In total:  1  node types.

Created 1 relationships from a 'Doctor' node
All relationships have been created successfully.



## Non-functional Tests - Usability Tests

### Get all illnesses

In [6]:
# define query to get all illness nodes
query_illness = """
MATCH (i:Illness)
RETURN i.name as name, i.ICDCode as ICD_Code, i.description as description
"""

# time the query execution
execution_time = timeit.timeit("graph.run(query_illness).data()", number=100, globals=globals())
# print the average execution time
print(f"Average execution time: {execution_time / 100} seconds")

# run the query and get the data of the result
result = graph.run(query_illness).data()

# print the details of the illness nodes
pd.DataFrame(result)

Average execution time: 0.000648008749994915 seconds


,name,ICD_Code,description
0,Acute Otitis Media,H66.9,Middle ear infection commonly seen in children...
1,Alcohol Use Disorder,F10.20,A chronic condition characterized by problemat...
2,Anterior Cruciate Ligament Tear,S83.51,A common knee injury in athletes involving lig...
3,Appendicitis,K35.80,Inflammation of the appendix often requiring s...
4,Atopic Dermatitis,L20.9,A chronic skin condition causing itchy and inf...
5,Attention-Deficit/Hyperactivity Disorder,F90.9,A neurodevelopmental disorder characterized by...
6,Basal Cell Carcinoma,C44.91,A common form of skin cancer that rarely metas...
7,Blunt Abdominal Trauma,S36.1,Injury to the abdominal organs from blunt forc...
8,Breast Cancer,C50.9,A malignant tumor originating in breast tissue.
9,Cataract,H25.9,Clouding of the eye\'s lens leading to decreas...


### Find all symptoms of a specific illness

In [7]:
# define query to get all symptoms of a specific illness
query = """ 
MATCH (s:Symptom)-[r:IS_SYMPTOM_OF]->(i:Illness) 
WHERE i.name = 'Migraine'
RETURN s.name as symptom
"""

print(graph)

test_query(query=query, graph=graph)

Graph('bolt://localhost:7687')
Average execution time: 0.0005956829199567438 seconds


,symptom
0,Light Sensitivity
1,Throbbing Headache


### Find all doctors who treated patients with a specific illness

In [8]:
# define query to get all symptoms of a specific illness
query = """ 
MATCH (d:Doctor)-[:TREATS]->(p:Patient)-[:HAS]->(i:Illness)
WHERE i.name = 'Breast Cancer'
RETURN DISTINCT d.name
"""

test_query(query=query, graph=graph)

Average execution time: 0.00023679334000917151 seconds


,d.name
0,Dr. Olivia Brown


### List illnesses that share at least one symptom

In [9]:
# define query to get all symptoms of a specific illness
query = """ 
MATCH (i1:Illness)<-[:IS_SYMPTOM_OF]-(s:Symptom)-[:IS_SYMPTOM_OF]->(i2:Illness)
WHERE i1.name <> i2.name
RETURN DISTINCT i1.name AS Illness1, i2.name AS Illness2, s.name AS SharedSymptom
"""

timeit.timeit("graph.run(query).data()", number=100, globals=globals())
print(f"Average execution time: {execution_time / 100} seconds")

result = graph.run(query).data()
pd.DataFrame(result)

Average execution time: 0.000648008749994915 seconds


,Illness1,Illness2,SharedSymptom
0,Prosthetic Joint Infection,Acute Otitis Media,Fever
1,Varicella (Chickenpox),Acute Otitis Media,Fever
2,Colorectal Cancer,Appendicitis,Abdominal Pain
3,Varicella (Chickenpox),Atopic Dermatitis,Itchy Rash
4,Appendicitis,Colorectal Cancer,Abdominal Pain
5,Pulmonary Embolism,Ischemic Heart Disease,Chest Pain
6,Acute Otitis Media,Prosthetic Joint Infection,Fever
7,Varicella (Chickenpox),Prosthetic Joint Infection,Fever
8,Ischemic Heart Disease,Pulmonary Embolism,Chest Pain
9,Acute Otitis Media,Varicella (Chickenpox),Fever


### Find patients allergic to drugs they were prescribed

In [10]:
# define query to get all symptoms of a specific illness
query = """ 
MATCH (p:Patient)-[:TAKES]->(d:Drug),
      (p)-[:HAS]->(a:Allergy)
WHERE d.name = a.name
RETURN p.name AS Patient, d.name AS ConflictMedicament
"""

timeit.timeit("graph.run(query).data()", number=100, globals=globals())
print(f"Average execution time: {execution_time / 100} seconds")

result = graph.run(query).data()
df = pd.DataFrame(result)

if len(df) == 0:
    print("Luckily, no patients have a conflict with their medication.")
else:
    display(df)

Average execution time: 0.000648008749994915 seconds
Luckily, no patients have a conflict with their medication.


### Find the most common symptom accross all illnesses

In [11]:
# define query to get all symptoms of a specific illness
query = """ 
MATCH (s:Symptom)-[:IS_SYMPTOM_OF]->(:Illness)
RETURN s.name, COUNT(*) AS Occurrence
ORDER BY Occurrence DESC
LIMIT 1
"""

timeit.timeit("graph.run(query).data()", number=100, globals=globals())
print(f"Average execution time: {execution_time / 100} seconds")

result = graph.run(query).data()
pd.DataFrame(result)

Average execution time: 0.000648008749994915 seconds


,s.name,Occurrence
0,Fever,3


## Restore database to original state

In [12]:
reset_db(graph=graph)

file = pd.ExcelFile(f"../data/export/export_{current_time}.xlsx")

excel_import(excel_file=file, graph=graph)

# delete the export file
os.remove(f"../data/export/export_{current_time}.xlsx")
print("Export file deleted.")
print(colored("--- Test completed successfully. ---", "green"))

Database reset completed.
Node worksheets:  ['Doctor', 'Topic', 'SubTopic', 'Illness', 'Symptom', 'Cause', 'Treatment', 'Patient', 'Drug', 'Diagnosis', 'Hospital', 'Allergy', 'Insurance', 'Department']
Relationship worksheets:  ['REL_Doctor', 'REL_Topic', 'REL_Illness', 'REL_Symptom', 'REL_Patient', 'REL_Hospital'] 

Created 10 nodes with the label 'Doctor'
Created 10 nodes with the label 'Topic'
Created 20 nodes with the label 'SubTopic'
Created 30 nodes with the label 'Illness'
Created 55 nodes with the label 'Symptom'
Created 55 nodes with the label 'Cause'
Created 55 nodes with the label 'Treatment'
Created 20 nodes with the label 'Patient'
Created 60 nodes with the label 'Drug'
Created 30 nodes with the label 'Diagnosis'
Created 5 nodes with the label 'Hospital'
Created 15 nodes with the label 'Allergy'
Created 5 nodes with the label 'Insurance'
Created 10 nodes with the label 'Department'
All nodes have been created successfully. In total:  14  node types.

Created 188 relationsh